In [283]:
from databases import PosicaoDm1,Crm,BDS
from funcoes_datas import FuncoesDatas
import pandas as pd
from datetime import datetime,date,timedelta
from dateutil.relativedelta import relativedelta
import re
import numpy as np 
from emailer import Email 

In [5]:
#Chamando as classesdo DataBases
posicao = PosicaoDm1()
crm= Crm()
fdt = FuncoesDatas()
bds = BDS()
fdt = FuncoesDatas()
hoje = fdt.hoje()

In [235]:
#Posicao em d1
carteira_atual = posicao.posicao_total_depara(hoje)[['DataArquivo','NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']]
carteira_atual=carteira_atual.sort_values(by=['NomeContaCRM','QuantidadeFinal','NomeProduto'])
carteira_atual['GuidProduto']= carteira_atual['GuidProduto'].str.lower()

#Retirar da carteira Caixa e Compromissada
carteira_atual = carteira_atual[(carteira_atual['TipoProduto']!='CAIXA')&(carteira_atual['TipoProduto']!='COMP')]

In [289]:
#Trazer todos os produtos e suas informações para complementar as decisões do comite

produtos = crm.rating_produtos()
produtos['GuidProduto']=produtos['GuidProduto'].str.lower()
produtos['CETIP']=produtos['CETIP'].str.lower()
produtos['ISIN']=produtos['ISIN'].str.lower()


In [290]:
#Verificar se nossa relação de ativos de venda estão coerentes com o call de crédito

#Verificação das Debs
aprovacao_credito_debs = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Debentures')[['CETIP','Status']]
aprovacao_credito_debs['CETIP'] = aprovacao_credito_debs['CETIP'].str.lower()
aprovacao_credito_debs = aprovacao_credito_debs[aprovacao_credito_debs['CETIP'].notnull()].dropna()
aprovacao_credito_debs = pd.merge(left=aprovacao_credito_debs,right=produtos,on='CETIP',how='left')[['GuidProduto','Status']].rename(columns={'Status':'Rating_Credito'})

#verificação dos FIDcs
aprovacao_credito_fidc = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='FIDC Recomendados')[['IdPAS']].astype('str')
aprovacao_credito_fidc = aprovacao_credito_fidc[aprovacao_credito_fidc['IdPAS'].notnull()].dropna()
aprovacao_credito_fidc = pd.merge(left=aprovacao_credito_fidc,right=produtos,on='IdPAS',how='left')[['GuidProduto']]
aprovacao_credito_fidc['Rating_Credito']='Recomendado'

#Bancos Aprovados
bancos_aprovados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Bancos JBFO')[['Banco']]
bancos_aprovados['Aprovacao']='Banco Aprovado'
bancos_aprovados['Banco'] = bancos_aprovados['Banco'].str.lower()

#Rating de crédito consolidado
rating_credito = pd.concat([aprovacao_credito_debs,aprovacao_credito_fidc]).dropna(subset=['GuidProduto'])


In [291]:
#Filtrando os ativos que foram deliberados no comite e trazendo as informações de nossa base de produtos

#Retorna os dados da planilha de forma bruta (trata os ISINS / CODs para localizar no CRM)
produtos_comite = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Consolidado - Ativos Analisados v3.xlsx')
produtos_comite['Cetip/Isin/CNPJ']=produtos_comite['Cetip/Isin/CNPJ'].str.lower()
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])
produtos_comite = produtos_comite[produtos_comite['On/Off']!='OFF']
produtos_comite = produtos_comite[produtos_comite['Deliberação'].str.contains('Zerar')]
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])

#Vincula os dados informados  com o codigo crm
cnpj = pd.merge(left =produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CNPJ',how='left')
isin =  pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='ISIN',how='left')
cetip = pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CETIP',how='left')

#Trazer todos os ativos que devem ser vendidos e verificando qual é o status de crédito
ativos_vender = pd.concat([cnpj,isin,cetip])
ativos_vender = pd.merge(left=ativos_vender,right=rating_credito,on='GuidProduto',how='left')
diferencas_credito = ativos_vender[ativos_vender['Rating_Credito']=='Recomendado']
diferencas_credito[['Data Comitê','Cliente','Ativo_y','Deliberação','Rating','Rating_Credito']]
diferencas_credito.to_excel('C:/Temp/Foundation/Verificar_ativos_diferentes_ratings.xlsx')

#Retornar ativos que não encontramos identificação para correção do cadastro e correção na planilha
ativos_sem_cod = ativos_vender[ativos_vender['GuidProduto'].isnull()]
ativos_sem_cod.to_excel('C:/Temp/Foundation/ativos_sem_cod.xlsx')

#Retirando os ativos que estão recomendados ou monitorados por crédito

ativos_vender = ativos_vender[(ativos_vender['GuidProduto'].notnull())&(ativos_vender['Rating_Credito']!='Recomendado')]
ativos_vender = ativos_vender[ativos_vender['Rating_Credito']!='Monitorado']

#Retorna apenas os clientes que passaram pelo comite e suas posições
string = carteira_atual['NomeContaCRM'].drop_duplicates().to_list()
substr = ativos_vender['Cliente'].drop_duplicates().to_list()


#Retorna todas as carteiras que foram aprovadas no comite (pelos nomes que foram aprovados)

def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]

carteiras = Filter(string, substr)
clientes_comite = carteira_atual.loc[carteira_atual['NomeContaCRM'].isin(carteiras)]
clientes_comite = clientes_comite.loc[clientes_comite['NomeContaCRM'].isin(carteiras)]
clientes_comite = pd.merge(left=clientes_comite,right=ativos_vender,on='GuidProduto',how='left').drop(columns=['Rating'])


#Retorna os dados dos produtos aprovados no comite
produtos_merge_comite = produtos[['GuidProduto','Rating','emissor','vencimento']]
clientes_comite = pd.merge(left=clientes_comite, right=produtos_merge_comite,on='GuidProduto',how='left').drop(columns=['emissor_x','vencimento_x'])

#Retorna o rating de créditos dos ativos
clientes_comite = pd.merge(left=clientes_comite,right=rating_credito,on='GuidProduto',how='left').drop(columns=['Rating_Credito_x']).rename(columns={'Rating_Credito_y':'Rating_Credito'})

#Tira tudo o que está como monitorado ou recomendado por crédito
clientes_comite = clientes_comite[(clientes_comite['Rating_Credito']!='Recomendado')&(clientes_comite['Rating_Credito']!='Monitorado')]
clientes_comite = clientes_comite[['DataArquivo','NomeContaCRM','GuidProduto','TipoProduto','NomeProduto','Rating','emissor_y','vencimento_y','QuantidadeFinal', 'FinanceiroFuturo','ISIN', 'CETIP','CNPJ','Data Comitê','Deliberação']]

#Ativos que tiveram correspondencia com o CRM e integrar com as posições dos clientes na D-1 - Verificar se ainda temos posição nele.
venda_dpms = clientes_comite[clientes_comite['Deliberação'].notnull()].drop_duplicates()

#ver os ativos reprovados que não passaram pelo comite
ativos_n_deliberados = clientes_comite[(clientes_comite['Deliberação'].isnull())&(clientes_comite['Rating']!='Recomendado')].drop_duplicates()
ativos_n_deliberados = ativos_n_deliberados[(ativos_n_deliberados['Rating']!='Monitorado')&(ativos_n_deliberados['TipoProduto']!='COE')]


#retorna só produtos bancarios e verifica se o banco é aprovado
produtos_bancos = ['LCA','CDB','LCI','LIG','LF','DPGE','LFS']
ativos_n_deliberados['emissor_y'] = ativos_n_deliberados['emissor_y'].str.lower()
bancarios = pd.merge(left=ativos_n_deliberados, right=bancos_aprovados,left_on='emissor_y',right_on='Banco',how='left')
bancarios = bancarios.loc[bancarios['TipoProduto'].isin(produtos_bancos)]
bancarios = bancarios[bancarios['Aprovacao']!='Banco Aprovado']
bancarios = bancarios[['NomeContaCRM','GuidProduto','NomeProduto','QuantidadeFinal']]
bancarios['Tipo_produto'] = 'Bancário'

#Ativos não bancarios que entraram na carteira sem deliberação
outros = ativos_n_deliberados.loc[~ativos_n_deliberados['TipoProduto'].isin(produtos_bancos)]
outros = outros[['NomeContaCRM','GuidProduto','NomeProduto','QuantidadeFinal']]
outros['Tipo_produto'] = 'Outros'

#Envia aos dpms os ativos que não foram deliberados e não são aprovados

ativos_n_deliberados = pd.concat([bancarios,outros]).reset_index(drop=True)
ativos_n_deliberados= pd.merge(left=ativos_n_deliberados, right=produtos,on='GuidProduto',how='left')
ativos_n_deliberados = ativos_n_deliberados[['NomeContaCRM','NomeProduto','QuantidadeFinal','ISIN','CETIP','emissor','vencimento']]


#Retornar todos os ativos que tivemos deliberações para venda e enviando para PM zerar
ativos_venda_pm = ativos_vender['GuidProduto'].to_list()
ativos_pm_vender = carteira_atual.loc[carteira_atual['GuidProduto'].isin(ativos_venda_pm)]
ativos_pm_vender = ativos_pm_vender[~ativos_pm_vender['NomeContaCRM'].str.contains('Monitorado')]
ativos_pm_vender = ativos_pm_vender[~ativos_pm_vender['NomeContaCRM'].str.contains('Recomendado')]



C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [293]:
# Envia email para a area das analises acima

#ativos_n_deliberados - ativos que entraram na carteira e precisam de aprovação do comite
#diferencas_credito - ativos aprovados por crédito que estão para zerar na origem
#ativos_sem_cod - não conseguimos localizar o ativo

#Deixar apenas as colunas importante
ativos_n_deliberados = ativos_n_deliberados#.drop(columns=['comite','Rating','IdPAS'])
diferencas_credito = diferencas_credito[['Data Comitê','Ativo_x','Deliberação','ISIN','CETIP','Rating_Credito']].rename(columns={'Ativo_x':'Ativo'})


#Gerando excel para e-mail:
with pd.ExcelWriter('C:/Temp/Foundation/Comite/Verificar_FLod.xlsx') as writer:  
    ativos_n_deliberados.to_excel(writer, sheet_name='ativos_n_deliberados')
    diferencas_credito.to_excel(writer, sheet_name='diferencas_deliberacao_credito')
    ativos_sem_cod.to_excel(writer, sheet_name='verificar_cadastro_ativo') 
    
#Envio do email

subject='[FLds] - Comitê de Exceção-Ajustes des ativos'
to=to = ['portfolio@juliusbaer.com','mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com','jbbr_im_risk@juliusbaer.com','fernando.pavicic@juliusbaer.com']
anexo = 'C:/Temp/Foundation/Comite/Verificar_FLod.xlsx'
text = '''Prezados,<br>

Favor verificar as inconsistências em nossas bases de ativos ou a aprovação dos ativos não deliberados.<br>
<br>
<br>
'''

email = Email(to = to , subject = subject, text= text,send = False, attachments = 'C:/Temp/Foundation/Comite/Verificar_FLod.xlsx')
